## Introdução
Conforme detalhei na Seção 4.2.3 da minha [monografia](https://github.com/anajbellini/SentimentAnalysis-MovieReviews/blob/master/documents/tcc-monografia-ajb.pdf), na etapa de processamento de texto do meu trabalho, defini 4 _pipelines_ distintos, sendo eles:

1. Aplicação de _stemming_, representando textos por _Bag-of-Words_;
2. Aplicação de _stemming_, representando textos por TF-IDF;
3. Aplicação de _lematização_, representando textos por _Bag-of-Words_; e
4. Aplicação de _lematização_, representando textos por TF-IDF.

A ordem de aplicação dos métodos pode ser vista abaixo, para melhor compreensão:

<center><img src="https://i.ibb.co/JxkJ377/fluxo.png" width="350"></center>

Neste Notebook, vou demonstrar os códigos que utilizei para cada método, para caso alguém esteja aprendendo Python e/ou processamento de linguagem natural e queira saber como fiz.

---

## Etapas Básicas
Não importa qual _pipeline_ estejamos usando, todos eles têm 3 etapas em comum: conversão para caixa baixa, remoção de pontuação e de _stop words_.

Para demonstrar o funcionamento destas três, usaremos o seguinte exemplo:

In [0]:
sentence = 'HelLo, wOrLd! How aRe YoU?'

### Caixa Baixa
Mesmo sendo uma etapa simples, é importante para que o classificador não entenda duas grafias de uma mesma palavra como sendo duas palavras distintas (por exemplo, `caixa` e `Caixa`).

A própria biblioteca String tem um método para fazer isso facilmente:

In [8]:
import string

lowercase = sentence.lower()

print(lowercase)

hello, world! how are you?


### Remoção de Pontuações
As pontuações não serão úteis para o aprendizado dos classificadores usados no trabalho, então todas são retiradas.

Especialmente para um dos _datasets_ que utilizei ([Large Movie Review Dataset](https://ai.stanford.edu/~amaas/data/sentiment/)), foi necessário incrementar esta etapa, removendo também _tags_ HTML que estavam presentes nos textos.

Com o auxílio dos métodos `translate()` e `maketrans()`, substituí todas as pontuações por espaços em branco.

In [9]:
tags_removed = lowercase.replace('<br />', ' ')
punctuation_removed = tags_removed.translate(tags_removed.maketrans(string.punctuation, ' ' * len(string.punctuation)))
                                             
print(punctuation_removed)

hello  world  how are you 


### Remoção de _Stop Words_
_Stop words_ são palavras que, sozinhas, não possuem significado. Elas sempre precisam estar acompanhadas de outras palavras para fazerem sentido. Alguns exemplos são artigos e preposições.

A biblioteca NLTK traz uma lista de _stop words_ para diversos idiomas (incluindo Português), que usamos para fazer a remoção delas de nossos conjuntos de dados, após separar cada texto em tokens. No meu trabalho, utilizei o conjunto de _stop words_ em Inglês.

In [14]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')

tokens = word_tokenize(punctuation_removed)
list_stopwords = set(stopwords.words('english'))
stopwords_removed = [word for word in tokens if word not in list_stopwords]

print(stopwords_removed)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
['hello', 'world']


Até este momento, os textos ficarão em forma de lista de tokens, para que sejam processadas posteriormente com a técnica escolhida. A partir daqui, os quatro _pipelines_ se dividem.

---
## Redução para Formas Básicas
Com o tratamento dos textos feito até aqui, a próxima etapa envolve a redução de todas as palavras do texto para alguma forma básica delas. Existem duas técnicas conhecidas, o _stemming_ e a lematização, que são aplicadas separadamente.

### _Stemming_
TODO

In [0]:
# TODO implementação do stemming

### Lematização
TODO

In [0]:
# TODO implementação da lematização

---
## Representação dos Textos
Por fim, para que o algoritmo de _Machine Learning_ consiga interpretar, extrair padrões do nosso conjunto de dados e gerar um modelo dele, é preciso que os textos sejam representados numericamente.

Dessa forma, utilizei _Bag-of-Words_ (BoW) e TF-IDF para fazer essa conversão.

### _Bag-of-Words_
TODO

In [0]:
# TODO implementação do BoW

### TF-IDF
TODO

In [0]:
# TODO implementação direta do TF-IDF, com TfidfVectorizer()

TODO

In [0]:
# TODO implementação do TF-IDF, em cima do Bag-of-Words

---

Com isso, encerra-se a implementação das técnicas de processamento de texto aplicadas no meu trabalho.

Para fins de modularização, para os experimentos do meu TCC, reuni todas estas técnicas em uma única classe Python, importando-a em meus Scripts dos modelos. Esta classe pode ser encontrada [aqui](https://github.com/anajbellini/SentimentAnalysis-MovieReviews/blob/master/models/text_processing.py).

Os códigos do meu trabalho (principalmente dos métodos demonstrados neste Notebook) ainda podem ser bastante melhorados. Por exemplo, um meio que estudei após a apresentação do meu trabalho envolve usar a classe `sklearn.pipeline.Pipeline`. Futuramente, farei uma nova versão dessas implementações do meu TCC.